In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from warnings import filterwarnings

In [3]:
## Data Ingestion step
df = pd.read_csv('https://raw.githubusercontent.com/krishnaik06/FSDSRegression/main/notebooks/data/gemstone.csv')

In [4]:
df.head(5)

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [5]:
df.drop(labels='id', axis=1, inplace=True)

In [6]:
## Separating Independent and dependent features from Dataset

X = df.iloc[:, 0:-1]
Y = df.iloc[:,-1]

In [7]:
X

,carat,cut,color,clarity,depth,table,x,y,z
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77
...,...,...,...,...,...,...,...,...,...
193568,0.31,Ideal,D,VVS2,61.1,56.0,4.35,4.39,2.67
193569,0.70,Premium,G,VVS2,60.3,58.0,5.75,5.77,3.47
193570,0.73,Very Good,F,SI1,63.1,57.0,5.72,5.75,3.62
193571,0.34,Very Good,D,SI1,62.9,55.0,4.45,4.49,2.81


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193573 entries, 0 to 193572
Data columns (total 10 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   carat    193573 non-null  float64
 1   cut      193573 non-null  object 
 2   color    193573 non-null  object 
 3   clarity  193573 non-null  object 
 4   depth    193573 non-null  float64
 5   table    193573 non-null  float64
 6   x        193573 non-null  float64
 7   y        193573 non-null  float64
 8   z        193573 non-null  float64
 9   price    193573 non-null  int64  
dtypes: float64(6), int64(1), object(3)
memory usage: 14.8+ MB


In [9]:
df[df.columns].dtypes == 'object'

carat      False
cut         True
color       True
clarity     True
depth      False
table      False
x          False
y          False
z          False
price      False
dtype: bool

In [10]:
# Define which columns should be ordinal-encoded and which should be scaled

categorical_columns = X.select_dtypes(include='object').columns
numerical_columns = X.select_dtypes(exclude='object').columns

In [11]:
categorical_columns

Index(['cut', 'color', 'clarity'], dtype='object')

In [35]:
numerical_columns

Index(['carat', 'depth', 'table', 'x', 'y', 'z'], dtype='object')

In [12]:
# Define the custom ranking for each ordinal variable. This is where Domain knowledge comes into picture. 

cut_categories = ['Fair', 'Good', 'Very Good','Premium','Ideal']
color_categories = ['D', 'E', 'F', 'G', 'H', 'I', 'J']
clarity_categories = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [13]:
## Libraries which we need to import for Feature engineering & Feature Scaling:

from sklearn.impute import SimpleImputer # This would help me to handle the missing values
from sklearn.preprocessing import StandardScaler # This would do Feature Scaling by subtracting the mean from the feature and then divide the result by feature std. deviation.
from sklearn.preprocessing import OrdinalEncoder # Ordinal Encoding

## Pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [14]:
## Numerical pipeline

num_pipeline = Pipeline(
    steps=[('imputer', SimpleImputer(strategy='median')),
           ('scaler', StandardScaler())]
)

## Categorical pipeline

cat_pipeline = Pipeline(
    steps=[('imputer', SimpleImputer(strategy='most_frequent')),
           ('ordinal_encoder', OrdinalEncoder(categories=[cut_categories, color_categories, clarity_categories])),
           ('scaler', StandardScaler())]
)

preprocessor = ColumnTransformer([
('num_pipeline', num_pipeline, numerical_columns),
('cat_pipeline', cat_pipeline, categorical_columns)
])

In [15]:
preprocessor

ColumnTransformer(transformers=[('num_pipeline',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 Index(['carat', 'depth', 'table', 'x', 'y', 'z'], dtype='object')),
                                ('cat_pipeline',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('ordinal_encoder',
                                                  OrdinalEncoder(categories=[['Fair',
                                                                              'Good',
                                                                              'Very '
                                                                              'Good',
                                                                              'Premium',
                                                                              'Ideal'],
                                                                             ['D',
                                                                              'E',
                                                                              'F',
                                                                              'G',
                                                                              'H',
                                                                              'I',
                                                                              'J'],
                                                                             ['I1',
                                                                              'SI2',
                                                                              'SI1',
                                                                              'VS2',
                                                                              'VS1',
                                                                              'VVS2',
                                                                              'VVS1',
                                                                              'IF']])),
                                                 ('scaler', StandardScaler())]),
                                 Index(['cut', 'color', 'clarity'], dtype='object'))])

In [16]:
## Train test split

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, random_state=40)

In [17]:
X_train.shape

(135501, 9)

In [18]:
X_test.shape

(58072, 9)

In [19]:
y_train.shape

(135501,)

In [20]:
y_test.shape

(58072,)

In [21]:
X_train = pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test = pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [22]:
X_test.head(2)

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,3.330432,0.1654,-0.637842,2.566503,2.532999,2.518156,0.871502,0.915044,-1.315348
1,-0.367702,0.1654,-0.117177,-0.228835,-0.207471,-0.194219,0.871502,-0.317026,-0.650314


In [23]:
X_train.head(2)

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,0.454106,-0.296545,-0.637842,0.636818,0.663672,0.622410,0.871502,-0.317026,-0.650314
1,3.027661,-1.589990,1.965484,2.494366,2.433180,2.182754,-0.133146,0.915044,-1.315348


In [24]:
## Model Training

from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [25]:
regression=LinearRegression()
regression.fit(X_train,y_train)

LinearRegression()

In [26]:
regression.coef_

array([ 6444.18494694,   -95.79974015,   -67.01577837, -1823.10963   ,
          -8.19306509,  -460.63495897,    74.53336566,  -464.42173169,
         650.12826018])

In [27]:
regression.intercept_

3966.0501029512675

In [28]:
import numpy as np
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mse, mae, rmse, r2_square

In [29]:
## Train multiple models

models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'Elasticnet':ElasticNet()
}

model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)

    #Make Predictions
    y_pred=model.predict(X_test)

    mse, mae, rmse, r2_square=evaluate_model(y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Training Performance')
    print("MSE:",mse)
    print("RMSE:",rmse)
    print("MAE:",mae)
    print("R2 score",r2_square*100)

    r2_list.append(r2_square)
    
    print('='*35)
    print('\n')

LinearRegression
Model Training Performance
MSE: 1034639.3993660724
RMSE: 1017.1722564866152
MAE: 674.6638217377892
R2 score 93.63431403063537


Lasso
Model Training Performance
MSE: 1031454.8814278347
RMSE: 1015.6056722113336
MAE: 675.3910275839803
R2 score 93.65390698366912


Ridge
Model Training Performance
MSE: 1034641.1428541328
RMSE: 1017.1731135131978
MAE: 674.6868170797178
R2 score 93.63430330371207


Elasticnet
Model Training Performance
MSE: 2362019.9950617175
RMSE: 1536.886461343751
MAE: 1065.5716610505156
R2 score 85.46751887552743




In [30]:
len(list(models))

4

In [31]:
list(models)

['LinearRegression', 'Lasso', 'Ridge', 'Elasticnet']

In [32]:
for i in range(len(list(models))):
    model=list(models.values())[i]
    print(model)

LinearRegression()
Lasso()
Ridge()
ElasticNet()


In [33]:
len(list(models))

4

In [34]:
for i in range(len(tuple(models))):
    model=tuple(models.keys())[i]
    print(model, i)

LinearRegression 0
Lasso 1
Ridge 2
Elasticnet 3
